Scrape the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 
in order to obtain the data that is in the table of postal codes.

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Only process the rows that have an assigned borough, and ignore the others

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

More than one neighborhood can exist in one postal code, and they should be combined into a single rows with the neighborhoods separated with a comma

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  58.06 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  37.01 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  41.12 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  50.22 MB/s


In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
html_doc = r.text

soup = BeautifulSoup(html_doc, 'lxml')
nb = soup.find_all('tbody')
nh = nb[0].find_all('tr')

columns= [col.find(text=True).replace('\n','') for col in nh[0].find_all('th')] 
data= [[col.find(text=True).replace('\n','') for col in row.find_all('td')] for row in nh[1:]]

df = pd.DataFrame(data, columns=columns)
df.rename(columns={'Neighbourhood':'Neighborhood', 'Postcode':'PostalCode'}, inplace=True)

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df.Borough

#more than one neighborhood can exist in one postal code area .. combine into one row with the neighborhoods separated with a comma
dfb = df.groupby('PostalCode')['Borough'].max().to_frame()
dfn = df.groupby('PostalCode')['Neighborhood'].apply(lambda x: ', '.join(x)).to_frame()

dfx = dfb.join(dfn)

dfx.reset_index(inplace=True)
dfx.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
dfx.shape

(103, 3)